In [7]:
# Add to environment variables PYTHONPATH = C:\Users\JFC_WIN\Documents\OpenSimula\src
import OpenSimula as osm
import pandas as pd

bestest_building_600 = {
    "name": "Bestest Building",
    "time_step": 3600,
    "n_time_steps": 8760,
    "initial_time": "01/01/2001 00:00:00",
    "components": [
        # MET_FILE
        {
            "type": "File_met",
            "name": "sevilla",
            "file_name": "met_files/sevilla.met"
        },
        # CONSTRUCTION
        {
            "type": "Material",
            "name": "Enlucido",
            "conductivity": 0.16,
            "density": 950,
            "specific_heat": 840
        },
        {
            "type": "Material",
            "name": "Aislamiento",
            "conductivity": 0.04,
            "density": 12,
            "specific_heat": 840
        },
        {
            "type": "Material",
            "name": "Madera pared",
            "conductivity": 0.14,
            "density": 530,
            "specific_heat": 900
        },
        {
            "type": "Material",
            "name": "Aislamiento suelo",
            "conductivity": 0.04,
            "density": .1,
            "specific_heat": .1,
        },
        {
            "type": "Material",
            "name": "Madera suelo",
            "conductivity": 0.14,
            "density": 650,
            "specific_heat": 1200,
        },
        {
            "type": "Material",
            "name": "Lecho_rocas",
            "conductivity": 0.14,
            "density": 650,
            "specific_heat": 1200,
        },
        {
            "type": "Construction",
            "name": "Pared 600",
            "solar_alpha": [0.6, 0.6],
            "materials": ["Enlucido", "Aislamiento", "Madera pared"],
            "thicknesses": [0.12, 0.066, 0.009],
        },
        {
            "type": "Construction",
            "name": "Suelo 600",
            "solar_alpha": [0.6, 0.6],
            "materials": ["Aislamiento suelo", "Madera suelo"],
            "thicknesses": [1.003, 0.025]
        },
        {
            "type": "Construction",
            "name": "Techo 600",
            "solar_alpha": [0.6, 0.6],
            "materials": ["Enlucido", "Aislamiento", "Madera pared"],
            "thicknesses": [0.010, 0.01118, 0.019],
        },
        # BUILDING
        {
            "type": "Building",
            "name": "Bestest 600",
            "file_met": "sevilla"
        },
        {
            "type": "Space_type",
            "name": "bestest space",
            "people_density": "0",
            "light_density": "0",
            "other_gains_density": "4.1667",
            "other_gains_radiant_fraction": 0.6,
            "infiltration": "0.5"
        },
        {
            "type": "Space",
            "name": "space_1",
            "building": "Bestest 600",
            "space_type": "bestest space",
            "floor_area": 48,
            "volume": 48*2.7,
        },
        # Surfaces
        {
            "type": "Exterior_surface",
            "name": "north_wall",
            "construction": "Pared 600",
            "space": "space_1",
            "ref_point": [8,6,0],
            "width": 8,
            "height": 2.7,
            "azimuth": 180,
            "altitude": 0
        },
        {
            "type": "Virtual_exterior_surface",
            "name": "east_wall",
            "space": "space_1",
            "ref_point": [8,0,0],
            "width": 6,
            "height": 2.7,
            "azimuth": 90,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "south_wall",
            "construction": "Pared 600",
            "space": "space_1",
            "ref_point": [0,0,0],
            "width": 8,
            "height": 2.7,
            "azimuth": 0,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "west_wall",
            "construction": "Pared 600",
            "space": "space_1",
            "ref_point": [0,6,0],
            "width": 6,
            "height": 2.7,
            "azimuth": -90,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "roof",
            "construction": "Techo 600",
            "space": "space_1",
            "ref_point": [0,0,2.7],
            "width": 8,
            "height": 6,
            "azimuth": 0,
            "altitude": 90
        },
        {
            "type": "Underground_surface",
            "name": "floor",
            "construction": "Suelo 600",
            "space": "space_1",
            "shape": "POLYGON",
            "ref_point": [0,0,0],
            "x_polygon": [0,8,8,0],
            "y_polygon": [0,0,6,6],
            "height": 6,
            "azimuth": 0,
            "altitude": -90
        },
    ],
}

In [8]:
sim = osm.Simulation()
pro = osm.Project("pro", sim)
pro.read_dict(bestest_building_600)
pro.simulate()

Reading project data from dictonary
Reading completed.
Checking project: Bestest Building
ok
Simulating Bestest Building: 10% 20% 30% 40% 50% 60% 70% 80% 90% 100%  End


In [9]:
pro.component("Bestest 600").KZ_matrix

array([[357.16720722]])

In [10]:
wall = "south_wall"
df_met = pro.component("sevilla").variable_dataframe()
df_space = pro.component("space_1").variable_dataframe()
df_wall = pro.component(wall).variable_dataframe()

df_graph = pd.DataFrame()
df_graph["date"] = df_met["date"]
df_graph["T_ext"] = df_met["temperature [°C]"]
df_graph["T_z"] = df_space["temperature [°C]"]
df_graph["T_s0"] = df_wall["T_s0 [°C]"]
df_graph["T_s1"] = df_wall["T_s1 [°C]"]

#pro.component("sevilla").variable_dataframe(frequency="m",value="max")
df_space.head(24)



,date,temperature [°C],humidity [g/kg],people_convective [W],people_radiant [W],people_latent [W],light_convective [W],light_radiant [W],other_gains_convective [W],other_gains_radiant [W],other_gains_latent [W],solar_direct_gains [W],infiltration_flow [m³/s],surfaces_convective [W],delta_int_energy [W],infiltration_heat [W]
0,2001-01-01 00:00:00,15.736087,0.0,0.0,0.0,0.0,0.0,0.0,80.00064,120.00096,0.0,0.000000,0.018,-79.953051,197.705750,-197.753339
1,2001-01-01 01:00:00,12.581779,0.0,0.0,0.0,0.0,0.0,0.0,80.00064,120.00096,0.0,0.000000,0.018,-93.396362,146.256439,-132.860717
2,2001-01-01 02:00:00,11.880824,0.0,0.0,0.0,0.0,0.0,0.0,80.00064,120.00096,0.0,0.000000,0.018,-182.081256,32.501317,69.579299
3,2001-01-01 03:00:00,11.277654,0.0,0.0,0.0,0.0,0.0,0.0,80.00064,120.00096,0.0,0.000000,0.018,-179.795848,27.967336,71.827873
4,2001-01-01 04:00:00,10.704643,0.0,0.0,0.0,0.0,0.0,0.0,80.00064,120.00096,0.0,0.000000,0.018,-179.872288,26.568883,73.302765
5,2001-01-01 05:00:00,10.169951,0.0,0.0,0.0,0.0,0.0,0.0,80.00064,120.00096,0.0,0.000000,0.018,-176.788552,24.792173,71.995739
6,2001-01-01 06:00:00,9.696040,0.0,0.0,0.0,0.0,0.0,0.0,80.00064,120.00096,0.0,0.000000,0.018,-173.401705,21.973929,71.427136
7,2001-01-01 07:00:00,9.270565,0.0,0.0,0.0,0.0,0.0,0.0,80.00064,120.00096,0.0,0.000000,0.018,-169.531626,19.728096,69.802889
8,2001-01-01 08:00:00,8.896345,0.0,0.0,0.0,0.0,0.0,0.0,80.00064,120.00096,0.0,0.000000,0.018,-165.344508,17.351558,67.992310
9,2001-01-01 09:00:00,8.923801,0.0,0.0,0.0,0.0,0.0,0.0,80.00064,120.00096,0.0,33.673766,0.018,-153.706970,-1.273081,74.979412


In [11]:
import plotly.express as px

fig = px.line(df_graph,x='date',y=['T_ext','T_z','T_s0','T_s1'],height=400)
fig.show()